# **Inferring**
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [21]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [22]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Product review text

In [23]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [24]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive. The reviewer is satisfied with the lamp, the customer service, and the company in general.


In [25]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


## Identify types of emotions

In [26]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


## Identify anger

In [27]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Extract product and company name from customer reviews

In [28]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [29]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


## Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.

In [30]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

Five topics discussed in the article are requested from the model in a format that each item is one or two words long and in a comma-separated list. ChatGPT returns the topics as government surveys, job satisfaction, NASA, etc.

In [31]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas without numbering them.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Government survey, Employee satisfaction, NASA, Social Security Administration, Job satisfaction


In [32]:
response.split(sep=', ')

['Government survey',
 'Employee satisfaction',
 'NASA',
 'Social Security Administration',
 'Job satisfaction']

## Make a news alert for certain topics

The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:

In [33]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [34]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as a dictionay where the key is a topic and the value is 0 or 1 for each topic if it appears.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [35]:
for i in response.split(', '):
    print(i)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [41]:
response = "{nasa: 1, local government: 0}" 


response = response.replace('{', '').replace('}', '').replace('\n', '').strip()


topic_dict = {i.split(': ')[0].strip(): int(i.split(': ')[1].strip()) for i in response.split(', ') if ': ' in i}


if topic_dict.get('nasa') == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [42]:
#Prompt 1: Inferring Sentiment

lamp_review = """
This lamp was very disappointing. It arrived late, missing parts, and the company support was unhelpful. I regret this purchase completely.
"""

prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print("Sentiment:", response)

Sentiment: The sentiment of the review is negative.


In [43]:
#Prompt 2: Identifying Emotions

prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print("Emotions:", response)


Emotions: disappointing, late, missing parts, unhelpful, regretful


In [44]:
#Prompt 3: Inferring Topics

story = """
A recent government survey revealed that public sector workers rated job satisfaction in various departments. NASA had the highest rating, while others struggled.
"""

topic_list = ["nasa", "public sector", "survey", "job satisfaction", "government policies"]

prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which is delimited \
with triple backticks.

Give your answer as a dictionary where the key is a topic \
and the value is 0 or 1 for each topic if it appears.

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print("Topics Dictionary:", response)


Topics Dictionary: {
    "nasa": 1,
    "public sector": 1,
    "survey": 1,
    "job satisfaction": 1,
    "government policies": 1
}


In [45]:
#Prompt 4: Extracting Information

prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Emotions expressed by the reviewer
- Key topic of discussion

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Emotions", and "Key Topic" as keys.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


{
    "Sentiment": "Negative",
    "Emotions": ["Disappointment", "Regret"],
    "Key Topic": "Product quality and customer support"
}


Prompts Tested:

-Sentiment detection.

-Emotions extraction.

-Topics detection using predefined topic lists.

-Multiple extractions in a single query (e.g., sentiment, emotions, topics).


Findings:

Successes:

-The model correctly inferred sentiments, emotions, and topics when prompts were clear and specific.

-For predefined topic lists, the dictionary format was accurate and easy to parse.

-Emotions were well-identified and matched the tone of the reviews.

Challenges:

-In some cases, emotions inferred by GPT included general terms like "worry" or "sadness," which were less relevant to the specific context of product reviews.

-For multi-task queries, responses were accurate but sometimes redundant (e.g., repeating "negative sentiment" when only the label was needed).

Learning Points:

-The quality of results improves significantly when the prompt is specific and structured (e.g., requesting JSON or lists).

-Using delimiters (like triple backticks) avoids ambiguities in the model’s understanding of the input text.

-For topic detection, predefined lists streamline results and ensure relevance to targeted themes.